# Exporting QCoDes Datasets

This notebook demonstrates how we can export QCoDeS datasets to other file formats. 

## Setup

First, we borrow an example from the measurement notebook to have some data to work with.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
from qcodes import load_or_create_experiment, initialise_database, Measurement
from qcodes.tests.instrument_mocks import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)

qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\jenielse\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\jenielse\.qcodes\logs\211025-29688-qcodes.log


In [2]:
# preparatory mocking of physical setup
dac = DummyInstrument("dac", gates=["ch1", "ch2"])
dmm = DummyInstrumentWithMeasurement("dmm", setter_instr=dac)
station = qc.Station(dmm, dac)

In [3]:
initialise_database()
exp = load_or_create_experiment(
    experiment_name="working_with_pandas", sample_name="no sample"
)

In [4]:
meas = Measurement(exp)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dac.ch2)  # register the second independent parameter
meas.register_parameter(
    dmm.v2, setpoints=(dac.ch1, dac.ch2)
)  # register the dependent one

We then perform a very basic experiment.

In [5]:
# run a 2D sweep

with meas.run() as datasaver:

    for v1 in np.linspace(-1, 0, 200, endpoint=False):
        for v2 in np.linspace(-1, 1, 201):
            dac.ch1(v1)
            dac.ch2(v2)
            val = dmm.v2.get()
            datasaver.add_result((dac.ch1, v1), (dac.ch2, v2), (dmm.v2, val))

dataset1 = datasaver.dataset

Starting experimental run with id: 27. 


## Export from comandline

The dataset can be exported using the `export` method. Currently exporting to netcdf and csv is supported.

In [6]:
dataset1.export("netcdf")

The `export_info` attribute contains information about where the dataset has been exported to:

In [7]:
dataset1.export_info

ExportInfo(export_paths={'nc': 'C:\\Users\\jenielse\\qcodes_15.nc'})

Looking at the signature of export we can see that in addition to the file format we can set the `prefix` and `path` to export to.

In [8]:
?dataset1.export

Signature:
dataset1.export(
    export_type: Union[qcodes.dataset.export_config.DataExportType, str, NoneType] = None,
    path: Union[str, NoneType] = None,
    prefix: Union[str, NoneType] = None,
) -> None
Docstring:
Export data to disk with file name {prefix}{run_id}.{ext}.
Values for the export type, path and prefix can also be set in the "dataset"
section of qcodes config.

Args:
    export_type: Data export type, e.g. "netcdf" or ``DataExportType.NETCDF``,
        defaults to a value set in qcodes config
    path: Export path, defaults to value set in config
    prefix: File prefix, e.g. ``qcodes_``, defaults to value set in config.

Raises:
    ValueError: If the export data type is not specified or unknown, raise an error
File:      c:\users\jenielse\source\repos\qcodes\qcodes\dataset\data_set.py
Type:      method


## Automatic export

Datasets may also be exported automatically using the configuration options given in dataset config section. 
Here you can toggle if a dataset should be exported automatically using the `export_automatic` option as well as set the default type, prefix and path. See [the table here](https://qcodes.github.io/Qcodes/user/configuration.html) for the relevant configuration options.

For more information about how to configure QCoDeS datasets see [the page about configuration](https://qcodes.github.io/Qcodes/user/configuration.html)  in the QCoDeS docs.